## Notebook for testing TFServing predictions (using wrapper class)

In [1]:
import sys
sys.path.append("../../rlmolecule/tfserving")
from tfserving_client import TFServingClient

In [2]:
# Prepare dataset for testing

import pandas as pd
from rdkit import Chem
import tensorflow as tf

import sys
sys.path.append('../../')
# Old import: from alphazero.preprocessor import preprocessor
from rlmolecule.molecule.policy.preprocessor import MolPreprocessor, load_preprocessor

preprocessor = load_preprocessor()

redox_df = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/redox_data.csv.gz').head(1280)
display(redox_df["ionization energy"])

test_dataset = tf.data.Dataset.from_generator(
    lambda: (preprocessor.construct_feature_matrices(Chem.MolFromSmiles(smiles), train=False) 
             for smiles in redox_df.smiles),
output_types=preprocessor.output_types,
output_shapes=preprocessor.output_shapes)\
.padded_batch(batch_size=128, padded_shapes=preprocessor.padded_shapes(),
padding_values=preprocessor.padding_values)

inputs = list(preprocessor.construct_feature_matrices(Chem.MolFromSmiles(smiles), train=False) 
              for smiles in redox_df.smiles)
inputs[0]

0            NaN
1            NaN
2            NaN
3       0.781466
4       0.313277
          ...   
1275         NaN
1276         NaN
1277    1.416731
1278    1.454662
1279    1.467125
Name: ionization energy, Length: 1280, dtype: float64

{'atom': array([2, 3, 4, 5, 5, 6, 7, 8, 8]),
 'bond': array([2, 2, 3, 3, 4, 3, 4, 3, 3, 5, 6, 3, 3, 3, 3, 3]),
 'connectivity': array([[0, 1],
        [1, 0],
        [1, 2],
        [2, 1],
        [2, 3],
        [2, 6],
        [3, 2],
        [3, 4],
        [4, 3],
        [4, 5],
        [5, 4],
        [6, 2],
        [6, 7],
        [6, 8],
        [7, 6],
        [8, 6]])}

In [3]:
help(TFServingClient)

Help on class TFServingClient in module tfserving_client:

class TFServingClient(builtins.object)
 |  TFServingClient(endpoint, model_name, default_port=8500, signature_name='serving_default')
 |  
 |  This is a class for interacting with TFServing. 
 |  
 |  This class allows establishing a chennel with a TFServing endpoint
 |  and using it for efficient predictions. 
 |  
 |  Args:
 |      endpoint (str): TFServing's endpoint; should be in the form: '<hostname>:<port>';
 |                      if port isn't specified, default port will be used.
 |      model_name (str): Specific model being queried.
 |      default_port (int): gRPC endpoint runs on port 8500 by default.
 |      signature_name (str): optional.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, endpoint, model_name, default_port=8500, signature_name='serving_default')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  predict(self, query_input, tensorize=True, timeout_s=10.0, layer_name

In [4]:
# Establish a channel for using TFServing

tfc = TFServingClient("r104u33", "reward") # arguments: hostname for TFServing and name of model

In [5]:
# Example of getting a single prediction from TFServing

result = tfc.predict(inputs[0])
print(result)

# Expected predictions:  
# float_val: 1.1218291521072388
# float_val: -0.05435873568058014

dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 1
  }
  dim {
    size: 2
  }
}
float_val: 1.1218291521072388
float_val: -0.05435870960354805

